# CH01.2. **Gradient Calculation**

## 00. **작업 환경 설정**

#### 00.0. **사전 변수 설정**

In [1]:
SEED_NUM = 2025

#### 00.1. **라이브러리 호출 및 옵션 설정**

In [2]:
#(1) Import libraries
import os
import random
import numpy as np
import torch

#(2) Set up options
os.environ['PYTHONHASHSEED'] = str(SEED_NUM)
random.seed(a=SEED_NUM)
np.random.seed(seed=SEED_NUM)
torch.use_deterministic_algorithms(mode=True)
torch.manual_seed(seed=SEED_NUM)

#### 00.2. **사용자정의함수 정의**

In [3]:
pass

#### 00.3. **클래스 정의**

In [4]:
pass

<b></b>

## 01. **자동 미분(Automatic Differentatiation)**

#### 01.1. **정의** : 복잡한 함수의 미분(접선의 기울기)을 자동으로 계산할 수 있게 도와주는 기능

#### 01.2. **동작 원리** :
##### $ \hspace{0.15cm} $ ① `requires_grad`가 설정된 텐서가 연산에 참여하게 되면, 내부적으로 연산 그래프(computation graph)를 구성
##### $ \hspace{0.15cm} $ ② 연산이 동작할 때마다 해당 연산에 대한 Backward 함수(연산의 역전파 정보)가 생성되어 각 텐서의 `grad_fn` 속성에 연결됨
##### $ \hspace{0.15cm} $ ③ 최종 연산에 대해 `.backward()` 메서드를 호출하면, 계산 그래프를 거슬러 올라가면서(Backprop) 기울기를 자동 연산해 `.grad` 속성에 갱신

<b></b>

## 02. **텐서의 자동미분과 그라디언트 계산**

<img src="../../img/01.2. Gradient Calculation (1).png" width="80%" height="80%"></img>

#### 03.1. **자동미분 준비** : `torch.tensor(···, requires_grad=True)`

In [5]:
#(1) Define `a`
a = torch.tensor(data=[2, 3], dtype=torch.float, requires_grad=True)

#(2) Print `a`
a

tensor([2., 3.], requires_grad=True)

In [6]:
#(3) Define `b`
b = a + 1 

#(4) Define `b`
b

tensor([3., 4.], grad_fn=<AddBackward0>)

In [7]:
#(5) Define `c`
c = b ** 2

#(6) Define `c`
c

tensor([ 9., 16.], grad_fn=<PowBackward0>)

In [8]:
#(5) Define `d`
d = 2 * c

#(6) Define `d`
d

tensor([18., 32.], grad_fn=<MulBackward0>)

In [9]:
#(7) Define `obj`
obj = d.mean()

#(8) Print `obj`
obj

tensor(25., grad_fn=<MeanBackward0>)

#### 03.2. **역전파 시행** : `torch.tensor.grad`

In [10]:
#(1) Perform backpropagation based on `obj`
obj.backward()

#(2) Print `a.grad`
a.grad

tensor([6., 8.])

##### **(`PLUS`)** $ \text{a.grad} = \frac{\partial{}\text{obj}}{\partial{}\textbf{a}} = \frac{\partial{}\text{obj}}{\partial{}\textbf{d}} \frac{\partial{}\textbf{d}}{\partial{}\textbf{c}} \frac{\partial{}\textbf{c}}{\partial{}\textbf{b}} \frac{\partial{}\textbf{b}}{\partial{}\textbf{a}} $